## Setting Up A JS Project
* Kotlin/JS uses gradle build system
* Kotlin/JS gradle plugin provides project configuration tools along with helper tasks for automating routine typical for javascript development



### Creating Kotlin/JS Project Using Gradle Script

* Add kotlin("js") Plugin to the build script 

``` kotlin
plugins{
     kotlin("js") version "1.3.70"
}

kotlin{
    // This section is used to configure project
}
```

* kotlin section can be used to manage following section of the project
    - Target Execution Environment: Browser or Node.js
    - Project Dependencies: Maven or NPM
    - Run Configuration
    - Test Configuration
    - Bundling for browser projects
    - Target Directory
    
### Choosing Execution Environment
* Kotlin/JS can choose 2 execution environment 
    - Browser for client-side scripting in browser
    - Node.js for running javascript code outside of the browser
    - To define the target we use
    ``` kotlin
        kotlin{
            target{
                browser{
                }
            }
        }
    ```

### Managing Dependencies
* kotlin/js support traditional gradle dependency declarations in the dependencies section of the build script
``` kotlin
    dependencies{
            implementation("org.example.myproject", "1.1.0")
    }
```

### NPM Dependencies
* The kotlin/js lets us declare npm dependencies along with other dependencies and does everything else automatically. It install the yarn package manager and uses it to download the dependencies from the NPM repository to the ```node_modules``` directory of your project
* To declare a NPM dependency, pass its name and version to the npm() function inside the dependency declaration

``` kotlin 
    dependencies{
        implementation(npm("react","16.12.0"))
    }
```

## Dynamic Type

Being a statically typed language, kotlin still has to be interoperate with untyped and loosely typed typed environment of javascript. To facilitate these use case, the dynamic type is available in the language.

`val dyn: dynamic = `

the `dynamic` type basically turns off kotlin type checker 
* A value of this type can be assigned to any variable or passed anywhere as the parameter.
* Any value can be assigned to a variable of type dynamic or passed to a function that takes dynamic as a parameter

**Note - On javascript platform the dynamic type are compiled as is**

A dynamic call always returns dynamic as a result, so we can chain such calls freely:

## Calling Javascript From Kotlin
We can freely talk to javascript using dynamic type, but if we want full power of kotlin type system, we need to create Kotlin headers for javascript library

### Inline Javascript

We can inline some of the javascript code using the `js("")` function 
``` kotlin
fun jsTypeOf(o: Any): String {
    return js("typeof o")
}
```

### External Modifier

To tell kotlin that a certain declaration is written in pure JavaScript, we should mark it `external` modifier. When the compiler sees such a declaration. it assumes that the implementation of the corresponding class, function or property is provided by the developer, and therefore doesn't try to generate any JavaScript code from the declaration. This means that we should omit bodies of external declaration. 

``` kotlin
    external fun alert(message:Any?): Unit

    external class Node{
        val firstChild: Node
        fun append(child: Node): Node
        fun removeChild(child: Node): Node
    }
    
    external val window: Window
```

**Note - `external` modifier is inherited by nested declaration**

The `external` modifier is only allowed on package-level declarations. you can't declare an `external` member on non-`external` class


### Declaring (static) members of a class

To define members of a class or on prototype in javascript we do
``` javascript
    fun MyClass() {...}
    MyClass.sharedMember = function(){}
```

But there is not such syntax in kotlin but it however does have a companion object. Kotlin treats companion objects of `external` class in a special way: instead of expecting an object, it assumes members of companion objects to be members of the class itself. 

``` kotlin
    external class MyClass{
        companion object {
            fun sharedMember()
        }
        
        fun ownMember()
    }
```

### Declaring optional Parameters.
An external function can have optional paramters. How the JavaScript implementation actually computes
default values of these parameters, is unknown to Kotlin, thus it's impossible to use the usual syntax to declare such parameters in kotlin, instead we should

``` kotlin
    external fun myFunWithOptionArgs(x:Int, y:String = definedExternally, z:Long = definedExternally)

```

### Extending Javascript Classes
``` kotlin
    external open class HTMLElement: Element(){
        
    }
    
    class CustomElement: HTMLElement(){
        fun foo(){
            alert("bar")
        }
    }
```

### external Interfaces
We can use interfaces in the scenario where a javascript function expect an object which has foo and bar methods.

``` kotlin
    external interface HasFooAndBar {
        fun foo()
        fun bar()
    }
    
    external fun myFunction(p: HasFooAndBar)
```

Another use case for external interfaces is to describe settings objects. For example:

``` kotlin
external interface JQueryAjaxSettings {
    var async: Boolean

    var cache: Boolean

    var complete: (JQueryXHR, String) -> Unit

    // etc
}

fun JQueryAjaxSettings(): JQueryAjaxSettings = js("{}")

external class JQuery {
    companion object {
        fun get(settings: JQueryAjaxSettings): JQueryXHR
    }
}

fun sendQuery() {
    JQuery.get(JQueryAjaxSettings().apply {
        complete = { (xhr, data) ->
            window.alert("Request complete")
        }
    })
}
```

## JavaScript Modules

Kotlin allows us to compile our kotlin project to Javascript modules for popular module system
1. Plain. Don't compile for any module system. As usual, you can access a module by its name in the global scope. This option is used by default.
2. Asynchronous Module Definition (AMD), which is in particular used by require.js library.
3. CommonJS convention, widely used by node.js/npm (require function and module.exports object)
4. Unified Module Definitions (UMD), which is compatible with both AMD and CommonJS, and works as "plain" when neither AMD nor CommonJS is available at runtime.



## JsModule annotation

To tell that an external class, package, function or property is a Javascript module, we can use @JsModule annotation. Consider you have the following CommonJs module called "hello"

``` javascript
module.exports.sayHello = function(name) { alert("Hello, " + name); }
```

``` kotlin
@JsModule("hello")
external fun sayHello(name: String)
```

### Applying @JsModule to packages

Some Javascript libraries export packages (namespaces) instead of function and classes. in terms of Javascript, it's an object that has members that are classes, function and properties. Importing these packages as Kotlin objects often looks unnatural. The compiler allows to map imported JavaScript packages to Kotlin packages. using the following notation.

``` kotlin
@file:JsModule("extModule")
package ext.jspackage.name

external fun foo()

external class c
```

### Importing deeper package hierarchies

In the previous example the JavaScript module exports a single package. However, some JavaScript libraries export multiple packages from within a module. This case is also supported by kotlin, Though you have to declare a new .kt file for each package you import


``` js
module.exports = {
    mylib: {
        pkg1: {
            foo: function() { /* some code here */ },
            bar: function() { /* some code here */ }
        },
        pkg2: {
            baz: function() { /* some code here */ }
        }
    }
}
```

Above js need to be changed to 
```kotlin
@file:JsModule("extModule")
@file:JsQualifier("mylib.pkg1")
package extlib.pkg1

external fun foo()

external fun bar()
```
and
``` kotlin
@file:JsModule("extModule")
@file:JsQualifier("mylib.pkg2")
package extlib.pkg2

external fun baz()
```


### @JsNonModule annotation
When a declaration has @JsModule, you can't use it from Kotlin code when you don't compile it to a javascript module. Usually developers distribute their libraries both JavaScript modules and downloadable `.js` files that you can copy to a project's static resources and include via script element. To tell kotlin that it's ok to use a @JsModule declaration from non-module environment. We should use @JsNonModule declaration.
``` js
function topLevelSayHello(name) { alert("Hello, " + name); }
if (module && module.exports) {
    module.exports = topLevelSayHello;
}
```
To

``` kotlin
@JsModule("hello")
@JsNonModule
@JsName("topLevelSayHello")
external fun sayHello(name: String)
```